In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.06G/1.06G [00:06<00:00, 181MB/s]
100% 1.06G/1.06G [00:06<00:00, 186MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    #  This parameter excludes the fully connected layers (classification head) from the VGG16 model.
    # The "top" of the network refers to the part responsible for classification. By setting this to False,
    # you're effectively removing the classification layers. This is often done when you
    #   want to use the convolutional base for feature extraction and add your own classification layers on top
    include_top = False,
    input_shape=(150,150,3)
)

In [ ]:
conv_base.summary();

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
model=Sequential();
model.add(conv_base),
model.add(Flatten()),
model.add(Dense(256,activation='relu')),
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary();

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


# In Convolutional Neural Networks (CNNs), data generators are a critical component of the training process.
#  They handle the efficient loading, preprocessing, and feeding of data to the neural network during training.
#   Data generators are especially important when dealing with large datasets that might not fit entirely
#   into memory.

# CNNs are commonly used in computer vision tasks like image classification, object detection, and segmentation

# , where input data often consists of images. Data generators help ensure that the model is trained on a
# diverse set of examples, they enable data augmentation, and they allow for efficient use of memory and
# processing resources.

In [ ]:
conv_base.trainable=False;

The code you provided is setting up data generators for a cat-vs-dog classification problem using the `image_dataset_from_directory`
 function. Based on the organization of your dataset into the main "train" and "test" folders,
  each containing "dog" and "cat" subfolders, here's what will happen when you apply this code:

1. **Training Data Generator (`train_ds`):**
   - The `image_dataset_from_directory` function will read images from the "train" directory.
   - It will create two classes: "dog" and "cat," automatically inferring these class names from the subfolder names.
   - It will integer-encode the classes: "dog" will be encoded as 0 and "cat" will be encoded as 1 (based on alphabetical order).
   - It will resize each image to (150, 150) pixels as specified by the `image_size` parameter.
   - The data generator will create batches of 32 images along with their corresponding integer-encoded labels.
   - Each batch will be loaded and provided to the model during training.

2. **Validation Data Generator (`validation_ds`):**
   - Similar to the training data generator, the `image_dataset_from_directory` function will read images from the "test" directory.
   - It will create the same two classes: "dog" and "cat."
   - It will integer-encode the classes in the same way as for the training data.
   - Images will be resized to (150, 150) pixels.
   - Batches of 32 images and their integer-encoded labels will be created for validation.

In summary, these data generators will take care of loading and preprocessing
the images from the directory structure you described. They will automatically organize the data into batches, handle label encoding, and provide the data to your neural network for both training and validation. This structure is suitable for training and evaluating a model for a cat-vs-dog classification problem.


In [ ]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [16]:
print(train_ds)

<_MapDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [17]:
model.summary();

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [18]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
 51/625 [=>............................] - ETA: 1:14:25 - loss: 0.7677 - accuracy: 0.5135

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()